In [1]:
import sys
import pathlib
sys.path.insert(0, str(pathlib.Path(
).resolve().parent)) #.. this has to be the folder reCNN_visual_prosthesis
from models import reCNN_bottleneck_CyclicGauss3d, reCNN_bottleneck_CyclicGauss3d_no_scaling, Lurz_Control_Model
import numpy as np
from Antolik_dataset import AntolikDataset, AntolikDataModule
from experiments.utils import pickle_read, download_model, get_neuron_estimates, reconstruct_orientation_maps, visualize_preferred_orientations, rotate_and_compute_distances, circular_distance
import pytorch_lightning as pl
import torch
from train_on_antolik import get_config
from model_trainer import Antolik_dataset_preparation_function, get_model

import torch.nn as nn
from pprint import pprint

In [2]:
model = download_model(artifact_name="reCNN_bottleneck_CyclicGauss3d:v966", model_class=reCNN_bottleneck_CyclicGauss3d_no_scaling)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: csng-cuni (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.15.8 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Global seed set to 42
/opt/conda/lib/python3.8/site-packages/neuralpredictors/measures/modules.py:78: UserWarning: Poissonloss is averaged per batch. It's recommended to use `sum` instead
  warnings.warn("Poissonloss is averaged per batch. It's recommended to use `sum` instead")
/opt/conda/lib/python3.8/site-packages/neuralpredictors/layers/cores/conv2d.py:128: UserWarning: The averaged value of regularizer will be used.
  warnings.warn("The averaged value of regularizer will be used.", UserWarning)


Model from ./artifacts/reCNN_bottleneck_CyclicGauss3d:v966/epoch=68-step=275999.ckpt loaded!


/auto/budejovice1/mpicek/reCNN_visual_prosthesis/readout.py:344: UserWarning: sigma is sampled from uniform distribution, instead of a fixed value. Consider setting fixed_sigma to True
  warnings.warn(


In [3]:
# setup the dataloader
config = get_config()
config["train_data_dir"] = "/storage/brno2/home/mpicek/reCNN_visual_prosthesis/data/antolik_reparametrized/one_trials.pickle"
config["test_data_dir"] = "/storage/brno2/home/mpicek/reCNN_visual_prosthesis/data/antolik_reparametrized/ten_trials.pickle"

dm = Antolik_dataset_preparation_function(config, load_data=True)

Data loaded successfully!
Loaded precomputed mean from /storage/brno2/home/mpicek/reCNN_visual_prosthesis/data/antolik_reparametrized/one_trials_mean.npy


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
print("Input shape without batch_size:", dm.get_input_shape())

input_tensor = torch.rand((config["batch_size"],) + dm.get_input_shape())
output_tensor = model(input_tensor)

print("Shape of output tensor:", output_tensor.shape)

Input shape without batch_size: torch.Size([1, 110, 110])
Shape of output tensor: torch.Size([15, 5000])


In [6]:
trainer = pl.Trainer(
    callbacks=[],
    gpus=[0],
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [7]:
test_score = trainer.test(model, dm.test_dataloader(), verbose=False)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-5acaaed3-7248-01f8-c341-4a1a6334c2ab]
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:452: UserWarning: Your `test_dataloader` has `shuffle=True`,it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

In [8]:
print(test_score)

[{'test/corr': 0.3362080454826355}]
